## train model phân loại nhãn của bình luận 

In [50]:
# Chạy 2 lệnh này nếu bạn chưa có thư viện
# !pip install nltk
# !pip install imbalanced-learn

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline # Sẽ dùng cho GridSearch

# Tải tài nguyên cho NLTK (chỉ cần chạy 1 lần)
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LINH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LINH\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\LINH\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [51]:
# cell 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import re
import string



# Giả sử đây là DataFrame của bạn (trong thực tế, bạn sẽ dùng pd.read_csv)
df = pd.read_csv('df_test.csv') 
# df = pd.DataFrame(data)

# Thêm dữ liệu giả định để mô phỏng sự mất cân bằng bạn đã nêu
# (Trong code thực tế, bạn không cần bước này, chỉ cần tải file của bạn)
more_data = {
    'airline_sentiment': ['negative']*917 + ['neutral']*309 + ['positive']*236,
    'negativereason': ['Late Flight']*300 + ['Bad Flight']*300 + ["Can't Tell"]*317 + [np.nan]*309 + [np.nan]*236,
    'airline': ['Other']*1462,
    'text': ['some negative text']*917 + ['some neutral text']*309 + ['some positive text']*236
}
df_more = pd.DataFrame(more_data)
df = pd.concat([df, df_more], ignore_index=True)

print("Tổng quan dữ liệu:")
print(df.info())
print("\nPhân bố cảm xúc:")
print(df['airline_sentiment'].value_counts(normalize=True) * 100)

Tổng quan dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16102 entries, 0 to 16101
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   airline_sentiment  16102 non-null  object
 1   negativereason     10095 non-null  object
 2   airline            16102 non-null  object
 3   text               16102 non-null  object
dtypes: object(4)
memory usage: 503.3+ KB
None

Phân bố cảm xúc:
airline_sentiment
negative    62.694075
neutral     21.165073
positive    16.140852
Name: proportion, dtype: float64


In [52]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
import re
import string

# Tải tài nguyên cho NLTK (chỉ cần chạy 1 lần nếu đã chạy rồi)
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

# Khởi tạo các đối tượng
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Bổ sung các từ có thể muốn loại bỏ
custom_stopwords = {'virginamerica', 'united', 'americanair', 'jetblue', 'usairways', 'southwestair'}
stop_words = stop_words.union(custom_stopwords)


def preprocess_text_advanced(text):
    """
    Hàm làm sạch văn bản (Nâng cao):
    1. Chuyển về chữ thường
    2. Loại bỏ URL
    3. Loại bỏ @mentions
    4. Loại bỏ dấu câu và số
    5. Tokenize (tách từ)
    6. Lemmatization (đưa về từ gốc)
    7. Loại bỏ Stopwords
    8. Ghép lại thành chuỗi
    """
    # Xử lý trường hợp đầu vào không phải là chuỗi (ví dụ: float, NaN)
    if not isinstance(text, str):
        return ""
        
    text = text.lower()  # Chuyển về chữ thường
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Loại bỏ URL
    text = re.sub(r'\@\w+', '', text)  # Loại bỏ @mentions
    text = text.translate(str.maketrans('', '', string.punctuation + string.digits))  # Loại bỏ dấu câu và số
    
    # Tokenize và Lemmatize
    tokens = text.split()
    
    # *** SỬA LỖI Ở ĐÂY ***
    # Đổi 'lemize' thành 'lemmatize'
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Loại bỏ stopwords
    filtered_tokens = [word for word in lemmatized_tokens if word not in stop_words and len(word) > 1]
    
    return " ".join(filtered_tokens)

# Giả sử 'df' là DataFrame gốc của bạn (bạn đã có df = pd.read_csv('df_test.csv'))
print("Đang áp dụng tiền xử lý nâng cao...")
df['text_cleaned'] = df['text'].apply(preprocess_text_advanced)
print("...Hoàn tất tiền xử lý.")

# In thử ví dụ sau khi làm sạch
print("\nVí dụ sau khi làm sạch (Nâng cao):")
# Kiểm tra xem df có đủ 4 dòng không
if len(df) > 3:
    print(f"Gốc: {df['text'].iloc[3]}")
    print(f"Sau khi làm sạch: {df['text_cleaned'].iloc[3]}")
else:
    print("Dữ liệu không đủ 4 dòng để in ví dụ iloc[3].")

Đang áp dụng tiền xử lý nâng cao...
...Hoàn tất tiền xử lý.

Ví dụ sau khi làm sạch (Nâng cao):
Gốc: @VirginAmerica it's really aggressive to blast obnoxious "entertainment" in your guests' faces &amp; they have little recourse
Sau khi làm sạch: really aggressive blast obnoxious entertainment guest face amp little recourse


In [53]:
# Giả sử X và y đã được tạo từ 'text_cleaned' mới
X = df['text_cleaned']
y = df['airline_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 1. Tạo một Pipeline
# Pipeline sẽ tự động thực hiện 2 bước:
# 'tfidf': Vector hóa văn bản
# 'svc': Chạy mô hình LinearSVC
pipeline_sentiment = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=10000)),
    ('svc', LinearSVC(class_weight='balanced', max_iter=2000, random_state=42, dual=False)) # dual=False nhanh hơn khi n_samples > n_features
])

# 2. Định nghĩa không gian tham số để tìm kiếm
# Chúng ta sẽ yêu cầu GridSearchCV thử 3 giá trị C
parameters_sentiment = {
    'svc__C': [0.1, 1, 10]  # Thử nghiệm với các giá trị regularization
    # Bạn có thể thêm nhiều tham số khác: 'tfidf__max_features': [5000, 10000, 20000]
}

# 3. Khởi tạo GridSearchCV
# cv=3 nghĩa là sử dụng 3-fold cross-validation
# n_jobs=-1 sử dụng tất cả CPU để tăng tốc
# scoring='f1_weighted' tối ưu theo F1-score
grid_search_sentiment = GridSearchCV(pipeline_sentiment, parameters_sentiment, cv=3, n_jobs=-1, scoring='f1_weighted', verbose=2)

print("\nBắt đầu huấn luyện và tìm kiếm tham số Mô hình 1...")
grid_search_sentiment.fit(X_train, y_train) # *** DÙNG DỮ LIỆU TRAIN CỦA BẠN ***
print("...Huấn luyện hoàn tất.")

# 4. Đánh giá mô hình tốt nhất
print(f"Tham số tốt nhất tìm được: {grid_search_sentiment.best_params_}")

best_model_sentiment = grid_search_sentiment.best_estimator_
y_pred = best_model_sentiment.predict(X_test) # *** DÙNG DỮ LIỆU TEST CỦA BẠN ***

print("\n--- Kết quả Mô hình 1 (Cải tiến với GridSearchCV) ---")
print(classification_report(y_test, y_pred))


Bắt đầu huấn luyện và tìm kiếm tham số Mô hình 1...
Fitting 3 folds for each of 3 candidates, totalling 9 fits
...Huấn luyện hoàn tất.
Tham số tốt nhất tìm được: {'svc__C': 0.1}

--- Kết quả Mô hình 1 (Cải tiến với GridSearchCV) ---
              precision    recall  f1-score   support

    negative       0.87      0.90      0.88      2019
     neutral       0.66      0.61      0.64       682
    positive       0.75      0.71      0.73       520

    accuracy                           0.81      3221
   macro avg       0.76      0.74      0.75      3221
weighted avg       0.80      0.81      0.81      3221



In [55]:
# lưu lại model 

import joblib

# Lưu model vào file .joblib
joblib.dump(best_model_sentiment, "best_model_sentiment.joblib")

print("Model đã được lưu thành công vào 'best_model_sentiment.joblib'.")


Model đã được lưu thành công vào 'best_model_sentiment.joblib'.


## train model tìm từ khóa chính trong bình luận 

In [54]:
# 1. Lọc dữ liệu (giống như trước)
df_neg = df[
    (df['airline_sentiment'] == 'negative') & 
    (df['negativereason'].notna()) & 
    (df['negativereason'] != "Can't Tell")
].copy()

if df_neg.empty:
    print("\nKhông có đủ dữ liệu `negativereason`.")
else:
    # 2. Xác định X và y (sử dụng 'text_cleaned' đã được cải tiến)
    X_reason = df_neg['text_cleaned']
    y_reason = df_neg['negativereason']

    # 3. Chia Train và Test
    X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reason, y_reason, test_size=0.2, random_state=42, stratify=y_reason)

    # 4. Vector hóa (Bước này PHẢI làm trước khi dùng SMOTE)
    # Chúng ta học từ vựng (fit) trên tập Train, và áp dụng (transform) cho cả Train và Test
    vectorizer_reason = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
    X_train_r_tfidf = vectorizer_reason.fit_transform(X_train_r)
    X_test_r_tfidf = vectorizer_reason.transform(X_test_r)

    print(f"Kích thước tập train (trước SMOTE): {X_train_r_tfidf.shape}")

    # 5. Áp dụng SMOTE (CHỈ trên tập training)
    smote = SMOTE(random_state=42, k_neighbors=3) # k_neighbors=3 vì có lớp chỉ có ~74 mẫu
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_r_tfidf, y_train_r)

    print(f"Kích thước tập train (sau SMOTE): {X_train_resampled.shape}")
    print("\nPhân bố nhãn sau SMOTE:")
    print(y_train_resampled.value_counts())

    # 6. Huấn luyện mô hình (dùng LinearSVC)
    # Lần này không cần class_weight='balanced' vì dữ liệu đã cân bằng rồi
    model_reason = LinearSVC(max_iter=2000, random_state=42, dual=False)
    
    print("\nBắt đầu huấn luyện Mô hình 2 (Cải tiến với SMOTE)...")
    model_reason.fit(X_train_resampled, y_train_resampled)
    print("...Huấn luyện hoàn tất.")

    # 7. Đánh giá mô hình (trên tập Test gốc, không SMOTE)
    y_pred_r = model_reason.predict(X_test_r_tfidf)

    print("\n--- Kết quả Mô hình 2 (Cải tiến với SMOTE) ---")
    print(classification_report(y_test_r, y_pred_r))

Kích thước tập train (trước SMOTE): (6870, 5000)
Kích thước tập train (sau SMOTE): (20952, 5000)

Phân bố nhãn sau SMOTE:
negativereason
Customer Service Issue         2328
Cancelled Flight               2328
Bad Flight                     2328
Late Flight                    2328
Flight Booking Problems        2328
Lost Luggage                   2328
Flight Attendant Complaints    2328
Damaged Luggage                2328
longlines                      2328
Name: count, dtype: int64

Bắt đầu huấn luyện Mô hình 2 (Cải tiến với SMOTE)...
...Huấn luyện hoàn tất.

--- Kết quả Mô hình 2 (Cải tiến với SMOTE) ---
                             precision    recall  f1-score   support

                 Bad Flight       0.47      0.61      0.53       176
           Cancelled Flight       0.68      0.70      0.69       169
     Customer Service Issue       0.73      0.66      0.70       582
            Damaged Luggage       0.75      0.40      0.52        15
Flight Attendant Complaints       0.37   

In [56]:
# lưu lại model 
import joblib

# Lưu model vào file .joblib
joblib.dump(model_reason, "model_reason.joblib")

print("Model đã được lưu thành công vào 'model_reason.joblib'.")


Model đã được lưu thành công vào 'model_reason.joblib'.
